# Adding Middleware to Agents

### Middleware allows you to intercept and modify agent interactions for logging, security, and other cross-cutting concerns.
### Middleware allows you to trace tokens, prevent infinite loop
### Your quality of Agentic Application in long term will depend on your middleware use.
### Here we will two middleware, but you can use as many you want as complex your Application Gets.
1. **Simple Logging Middleware**
2. **Middleware to log functions**

In [23]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre
# You can see the basic_food_agent.ipynb for basic agent run

### 1. Simple Logging Middleware to log Agent's Run 

In [29]:
from agent_framework import AgentRunContext

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Simple middleware that logs agent execution."""
    print("Agent starting...")

    # Continue to agent execution
    await next(context)

    print("Agent finished!")

### 2. Function Logging Middleware

In [30]:

# Here we add the logging Function middleware since we are working with functions
from agent_framework import AgentRunContext, FunctionInvocationContext
from typing import Callable , Awaitable

# Function middleware 
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"Calling function: {context.function.name}")

    await next(context)

    print(f"Function result: {context.result}")


<h4> Here we will use meals db API from where we can get the random meal and data</h4>

In [31]:
import requests
import json
from typing import Annotated, List, Dict, Any, Optional
from pydantic import Field

# Clean the MealDB response for the LLM
def _clean_meal_data(meal: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper function to restructure the raw meal API response into a clean, 
    LLM-friendly format by combining ingredients and measures.
    """
    if not meal:
        return {}

    # Combine ingredients and measures into a single list
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append(f"{measure.strip()} {ing.strip()}".strip())

    return {
        "id": meal.get("idMeal"),
        "name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "ingredients": ingredients,
        "tags": meal.get("strTags"),
        "youtube_link": meal.get("strYoutube")
    }

# Get random meal for today
def get_random_meal() -> str:
    """
    Retrieves a random meal recipe from the database. 
    Useful when the user wants a surprise suggestion or explicitly asks for a random recommendation.

    Returns:
        str: A JSON string containing the meal name, ingredients, and cooking instructions.
    """
    try:
        response = requests.get("https://www.themealdb.com/api/json/v1/1/random.php")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"error": "No meal found."})
            
        meal = _clean_meal_data(data["meals"][0])
        return json.dumps(meal, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch random meal: {str(e)}"})

# Here we get the meal by name
def get_meal_by_name(
    meal_name: Annotated[str, Field(description="The name of the meal to search for (e.g., 'Arrabiata', 'Burger').")]
) -> str:
    """
    Searches for a specific meal recipe by name. 
    Use this when the user asks for a specific dish or wants to know how to cook a named item.

    Args:
        meal_name: The name of the dish to search for.

    Returns:
        str: A JSON string containing a list of matching meals with their details.
    """
    try:
        # The API requires a search query parameter 's'
        response = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"status": "not_found", "message": f"No meals found with the name '{meal_name}'."})
        
        # Clean and limit results (e.g., top 3 matches to save tokens)
        results = [_clean_meal_data(m) for m in data["meals"][:3]]
        return json.dumps(results, indent=2)

    except Exception as e:
        return json.dumps({"error": f"Failed to search for meal: {str(e)}"})

In [32]:
# Import core dependencies to create the agent, for Agent Framework
import asyncio
import os
import json

from dotenv import load_dotenv, find_dotenv
# Core components for building Agent, tool-enabled agents
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [33]:
# load environment file
load_dotenv(find_dotenv())

True

In [34]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("OPENROUTER_ENDPOINT"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    model_id="nvidia/nemotron-3-nano-30b-a3b:free"
)

<h2> Here we configure the System Prompt for Agent with Name</h2>

In [35]:
AGENT_NAME = "FoodAgent"

AGENT_INSTRUCTIONS = """You are an expert AI Chef dedicated to helping users discover and prepare delicious meals.

CORE BEHAVIORS:
1. **Tool Usage**: You have access to a recipe database. ALWAYS use the provided tools to answer questions about recipes. Do not guess or hallucinate ingredients.
   - Use `get_meal_by_name` when the user asks for a specific dish.
   - Use `get_random_meal` when the user is undecided, asks for a suggestion, or wants a surprise.

2. **Response Format**: 
   - Start with an appetizing description of the dish.
   - List key ingredients clearly (based on the tool output).
   - Summarize the cooking instructions to be easy to follow.
   - If the tool provides a YouTube link, always include it at the end.
   - Include Tool Name used for fetching response

3. **Constraints**: 
   - Keep your response friendly but strictly under 200 words. 
   - If instructions are long, summarize the key steps to fit the word limit.
"""

### Option1 -> We can add the middleware while creating the AGent.

In [36]:
# create the agent remember we are not using any tools here, this is simple example
agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal, get_meal_by_name],
    middleware=[logging_agent_middleware,logging_function_middleware]
)

#### You can clearly see the Agent's Logging while execution and Function Execution

In [39]:
response1 = await agent.run("Help me something to eat today?")

Agent starting...
Calling function: get_random_meal
Function result: {
  "id": "53025",
  "name": "Ful Medames",
  "category": "Vegetarian",
  "area": "Egyptian",
  "instructions": "As the cooking time varies depending on the quality and age of the beans, it is good to cook them in advance and to reheat them when you are ready to serve. Cook the drained beans in a fresh portion of unsalted water in a large saucepan with the lid on until tender, adding water to keep them covered, and salt when the beans have softened. They take 2\u20132 1/2 hours of gentle simmering. When the beans are soft, let the liquid reduce. It is usual to take out a ladle or two of the beans and to mash them with some of the cooking liquid, then stir this back into the beans. This is to thicken the sauce.\r\nServe the beans in soup bowls sprinkled with chopped parsley and accompanied by Arab bread.\r\nPass round the dressing ingredients for everyone to help themselves: a bottle of extra-virgin olive oil, the quar

In [40]:
print(response1.text)

**Ful Medames – Creamy Egyptian Bean Bowl**  
A comforting, protein‑rich dish of slow‑cooked broad beans mashed into a silky sauce, brightened with lemon, parsley, and a drizzle of olive oil. It’s perfect for a satisfying lunch or a hearty breakfast.

**Key Ingredients**  
- 2 cups broad (fava) beans  
- 1/3 cup chopped fresh parsley  
- 3 lemons (quartered)  
- 4 garlic cloves, crushed  
- Olive oil, cumin, salt & pepper  
- Optional toppings: boiled eggs, cucumber‑tomato salad, pickles

**Quick Instructions**  
1. Simmer the beans in unsalted water, lid on, for 2–2½ hours until tender, adding water as needed.  
2. When soft, mash a handful of beans with some cooking liquid and stir back in to thicken the sauce.  
3. Season with salt, pepper, and a splash of olive oil.  
4. Serve in bowls, garnish with parsley, lemon wedges, and your choice of toppings.  

**Watch the full preparation:** https://www.youtube.com/watch?v=ixpCabILuxw  

*Tool used: get_random_meal*


#### There are many other complex ways to include agents, which we will cover later 